In [77]:
from seleniumwire import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import json
from generic_methods import getLocationUrl
from selenium.webdriver.common.by import By

# pip install webdriver-manager
# pip install selenium-wire
# pip install bs4
# pip install pandas


In [78]:
locations = json.load(open("locations.json", "r"))

chrome_options = webdriver.ChromeOptions()
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.javascript": 2
}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chrome_options=chrome_options)


In [79]:
url = f"https://www.tripadvisor.com.ph/{locations['Manila']['attractions']}"
NUM_PAGES = 10 # number of pages to scrape.  total attractions scraped will be NUM_PAGES * 30 
offset = 0
attraction_links = []

for i in range(NUM_PAGES):
    index = url.index('-g')
    offset_url = url[:index] + f"-oa{offset}" + url[index:]
    driver.get(offset_url)
    attraction_list_page = BeautifulSoup(driver.page_source, "html.parser")

    attraction_items = []
   
    attraction_items.extend(attraction_list_page.find_all("div", class_="alPVI eNNhq PgLKC tnGGX"))

    for item in attraction_items:
        attraction_links.append(item.contents[0].get("href"))

    offset += 30

print(attraction_links)

['/Attraction_Review-g298573-d548076-Reviews-Intramuros-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d586732-Reviews-Fort_Santiago-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d310892-Reviews-San_Agustin_Church-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d310891-Reviews-Rizal_Park-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d310883-Reviews-Manila_Cathedral-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d310887-Reviews-National_Museum-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d549326-Reviews-Museo_San_Agustin-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d1157085-Reviews-Robinsons_Place_Mall-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d1137283-Reviews-Manila_Ocean_Park-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d554239-Reviews-Manila_Bay-Manila_Metro_Manila_Luzon.html', '/Attraction_Review-g298573-d549830-Reviews-Divisoria_Market-Manila_Met

In [80]:
def getAttractionInfo(page: BeautifulSoup, url):
    attraction_info = { }
    # print(attraction_url)
    driver.get(url)

    attraction_info = {}
    page = BeautifulSoup(driver.page_source, "html.parser")

    attraction_info["name"] = page.find("div", class_="iSVKr").contents[0].contents[0]
    attraction_info["url"] = url
    try:
        attraction_info["about"] = page.find("div", class_="pqqta _d").contents[0].contents[0].contents[0].contents[0]
    except:
        pass
    try:
        attraction_info["site_operating_hours"] = page.find("span", class_="EFKKt").contents[0]
    except:
        pass
    try:
        attraction_info["suggested_trip_duration"] = page.find("div", class_="nvXSy f _Y Q2").contents[0].contents[1].contents[0].contents[0]
    except:
        pass
    try:
        attraction_info["site_type"] = page.find_all("div", class_="kUaIL")[2].contents[0].contents[0].contents[0].contents[0].replace(" • ",",")
    except:
        pass

    try:
        review_info = page.find_all("div", class_="jVDab o W f u w GOdjs")[1]
        attraction_info["rating"] = float(review_info.contents[0].get("aria-label").split(" ")[0])
        attraction_info["review_count"] = int(review_info.contents[1].contents[0].split(" ")[0].replace(",",""))
        attraction_info["rating_description"] = getRatingDescription(attraction_info["rating"])
        
    except:
        pass

    prices = []
    for price_element in page.find_all("div", class_="biGQs _P fiohW avBIb fOtGX"):
        try:
            prices.append(float(price_element.contents[0].replace("₱","").replace(",","")))
        except:
            continue
    attraction_info["entrance_fee"] = sum(prices)/len(prices) if len(prices) > 0 else None

    try:
        attraction_info["address"] = page.find("div", class_="AcNPX A").contents[0].contents[0].contents[0].contents[0].contents[0].contents[1].contents[0].contents[0]
    except:
        pass

    attraction_reviews = []


    for review_elements in page.find("div","LbPSX").contents[0].contents:
        try:
            review = {}
            review["link"] = review_elements.find_all("a",class_="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS")[1].get("href")
            review["title"] = review_elements.find_all("a", class_="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS")[1].contents[0].contents[0]
            review["user_link"] = review_elements.find_all("a",class_="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS")[0].get("href")
            review["user_name"] = review_elements.find_all("a", class_="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS")[0].contents[0]

            review["rating"] = review_elements.find("svg","UctUV d H0").get("aria-label").split(" ")[0]
            review["content"] = review_elements.find("div","biGQs _P pZUbB KxBGd").contents[0].contents[0]

            addtl_details = review_elements.find("div", class_="RpeCd").contents[0].split(" • ")
            review["visit_date"] = addtl_details[0]
            review["purpose"] = addtl_details[1] if len(addtl_details) >1 else None

            review["review_date"] = review_elements.find("div", class_="TreSq").contents[0].contents[0].replace("Written ",",")
        except:
            pass

        attraction_reviews.append(review)
    return attraction_info, attraction_reviews

In [86]:
attraction_infos = []
attraction_reviews = []
for link in attraction_links:
    try:
        attraction_url = "https://www.tripadvisor.com.ph" + link
        print(attraction_url)
        driver.get(attraction_url)

        attraction_info, attraction_review = getAttractionInfo( BeautifulSoup(driver.page_source, "html.parser") , attraction_url)

        attraction_infos.append(attraction_info)
        attraction_reviews.extend(attraction_review)
    except Exception as e:
        print(f"An error has occured on {attraction_url}:{e}")
        continue


pd.DataFrame(attraction_infos).to_csv("datasets/attraction_infos.csv", index=False)
pd.DataFrame(attraction_reviews).to_csv("datasets/attraction_reviews.csv", index=False)
    # break
    

https://www.tripadvisor.com.ph/Attraction_Review-g298573-d548076-Reviews-Intramuros-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attraction_Review-g298573-d586732-Reviews-Fort_Santiago-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attraction_Review-g298573-d310892-Reviews-San_Agustin_Church-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attraction_Review-g298573-d310891-Reviews-Rizal_Park-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attraction_Review-g298573-d310883-Reviews-Manila_Cathedral-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attraction_Review-g298573-d310887-Reviews-National_Museum-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attraction_Review-g298573-d549326-Reviews-Museo_San_Agustin-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attraction_Review-g298573-d1157085-Reviews-Robinsons_Place_Mall-Manila_Metro_Manila_Luzon.html
https://www.tripadvisor.com.ph/Attractio